Below is a consolidated script that addresses all the questions using Python and Keras. This script covers the generation of images using DCGAN, fine-tuning a ResNet50 on CIFAR-10, and implementing a GAN from scratch to generate celebrity faces.


import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import cifar10
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, Reshape, Conv2D, Conv2DTranspose, LeakyReLU, Dropout, BatchNormalization, Input, Softmax
from keras.optimizers import Adam
from keras.applications import ResNet50
from keras.losses import BinaryCrossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras import initializers
from keras.layers import GlobalAveragePooling2D
import keras

# Question 1: DCGAN for Image Generation from Noise

def build_generator():
    model = Sequential()
    model.add(Dense(256*4*4, activation="relu", input_dim=100))
    model.add(Reshape((4, 4, 256)))
    model.add(Conv2DTranspose(128, kernel_size=4, strides=2, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2DTranspose(64, kernel_size=4, strides=2, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2DTranspose(3, kernel_size=4, strides=2, padding="same", activation='tanh'))
    return model

def build_discriminator():
    model = Sequential()
    model.add(Conv2D(64, kernel_size=4, strides=2, input_shape=(32, 32, 3), padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(128, kernel_size=4, strides=2, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    return model

def train_dcgan(generator, discriminator, combined, epochs=10000, batch_size=64, save_interval=1000):
    (X_train, _), (_, _) = cifar10.load_data()
    X_train = (X_train.astype(np.float32) - 127.5) / 127.5

    valid = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    for epoch in range(epochs):
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        imgs = X_train[idx]

        noise = np.random.normal(0, 1, (batch_size, 100))
        gen_imgs = generator.predict(noise)

        d_loss_real = discriminator.train_on_batch(imgs, valid)
        d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        g_loss = combined.train_on_batch(noise, valid)

        if epoch % save_interval == 0:
            print(f"{epoch} [D loss: {d_loss[0]} | D accuracy: {100*d_loss[1]}] [G loss: {g_loss}]")
            save_images(epoch, generator)

def save_images(epoch, generator, examples=10, dim=(1, 10), figsize=(10, 1)):
    noise = np.random.normal(0, 1, (examples, 100))
    generated_images = generator.predict(noise)
    generated_images = 0.5 * generated_images + 0.5

    plt.figure(figsize=figsize)
    for i in range(examples):
        plt.subplot(dim[0], dim[1], i+1)
        plt.imshow(generated_images[i])
        plt.axis('off')
    plt.tight_layout()
    plt.savefig(f"gan_generated_image_epoch_{epoch}.png")
    plt.close()

generator = build_generator()
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5), metrics=['accuracy'])

z = Input(shape=(100,))
img = generator(z)
discriminator.trainable = False
valid = discriminator(img)

combined = Model(z, valid)
combined.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5))

train_dcgan(generator, discriminator, combined)

# Question 2: Fine-tuning ResNet50 on CIFAR-10

def fine_tune_resnet50(num_classes, strategy):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
    
    if strategy == 'from_scratch':
        for layer in base_model.layers:
            layer.trainable = True
    elif strategy == 'freeze_base':
        for layer in base_model.layers:
            layer.trainable = False
    elif strategy == 'fine_tune_all':
        for layer in base_model.layers:
            layer.trainable = True
    
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(512, activation='relu')(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)
datagen.fit(x_train)

strategies = ['from_scratch', 'freeze_base', 'fine_tune_all']
for strategy in strategies:
    print(f"Training with strategy: {strategy}")
    model = fine_tune_resnet50(10, strategy)
    history = model.fit(datagen.flow(x_train, y_train, batch_size=64), epochs=50, validation_data=(x_test, y_test))
    plt.plot(history.history['accuracy'], label=f'{strategy}_train_acc')
    plt.plot(history.history['val_accuracy'], label=f'{strategy}_val_acc')

plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(loc='upper left')
plt.show()

# Question 3: GAN for Celebrity Faces using Keras

from keras.preprocessing.image import ImageDataGenerator

def build_gan(generator, discriminator):
    z = Input(shape=(100,))
    img = generator(z)
    discriminator.trainable = False
    valid = discriminator(img)
    combined = Model(z, valid)
    combined.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5))
    return combined

def train_gan(generator, discriminator, combined, data_generator, epochs=10000, batch_size=64, save_interval=1000):
    valid = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    for epoch in range(epochs):
        imgs = data_generator.next()
        imgs = (imgs - 127.5) / 127.5

        noise = np.random.normal(0, 1, (batch_size, 100))
        gen_imgs = generator.predict(noise)

        d_loss_real = discriminator.train_on_batch(imgs, valid)
        d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        g_loss = combined.train_on_batch(noise, valid)

        if epoch % save_interval == 0:
            print(f"{epoch} [D loss: {d_loss[0]} | D accuracy: {100*d_loss[1]}] [G loss: {g_loss}]")
            save_images(epoch, generator)

celeba_datagen = ImageDataGenerator(rescale=1./255).flow_from_directory(
    'path/to/celeba/img_align_celeba',
    target_size=(32, 32),
    batch_size=64,
    class_mode=None
)

gan_generator = build_generator()
gan_discriminator = build_discriminator()
gan_discriminator.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5), metrics=['accuracy'])
gan_combined = build_gan(gan_generator, gan_discriminator)

train_gan(gan_generator, gan_discriminator, gan_combined, celeba_datagen)

# Note: This script assumes the path to the CelebA dataset is correctly specified.


CODING ANSWERS-

import tensorflow as tf
from tensorflow.keras.layers import Dense, Conv2D, Conv2DTranspose, Flatten, Reshape, LeakyReLU, BatchNormalization, Dropout
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt

# 1. Data Augmentation Function for GAN Training
def augment_data(dataset, rotation_range=20, flip_prob=0.5, crop_size=(32, 32)):
    datagen = ImageDataGenerator(
        rotation_range=rotation_range,
        horizontal_flip=True,
        width_shift_range=0.1,
        height_shift_range=0.1,
        zoom_range=0.2
    )
    augmented_data = datagen.flow(dataset, batch_size=len(dataset), shuffle=False)
    return next(augmented_data)[0]

# 2. Create a Simple Discriminator Model
def build_discriminator(input_shape=(32, 32, 3)):
    model = Sequential([
        Conv2D(64, kernel_size=4, strides=2, input_shape=input_shape, padding="same"),
        LeakyReLU(alpha=0.2),
        Dropout(0.25),
        Conv2D(128, kernel_size=4, strides=2, padding="same"),
        LeakyReLU(alpha=0.2),
        Dropout(0.25),
        Flatten(),
        Dense(1, activation='sigmoid')
    ])
    return model

# 3. Create a Generator Model
def build_generator(latent_dim=100):
    model = Sequential([
        Dense(256*4*4, activation="relu", input_dim=latent_dim),
        Reshape((4, 4, 256)),
        Conv2DTranspose(128, kernel_size=4, strides=2, padding="same"),
        BatchNormalization(momentum=0.8),
        LeakyReLU(alpha=0.2),
        Conv2DTranspose(64, kernel_size=4, strides=2, padding="same"),
        BatchNormalization(momentum=0.8),
        LeakyReLU(alpha=0.2),
        Conv2DTranspose(3, kernel_size=4, strides=2, padding="same", activation='tanh')
    ])
    return model

# 4. Minimax Loss Function for GANs
def minimax_loss(real_output, fake_output):
    cross_entropy = BinaryCrossentropy(from_logits=True)
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

# 5. GAN Model Combining Generator and Discriminator
def build_gan(generator, discriminator):
    discriminator.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    discriminator.trainable = False

    gan_input = tf.keras.Input(shape=(100,))
    generated_image = generator(gan_input)
    gan_output = discriminator(generated_image)
    gan = Model(gan_input, gan_output)
    gan.compile(loss='binary_crossentropy', optimizer='adam')

    return gan

# Training Function for GAN
def train_gan(generator, discriminator, gan, dataset, epochs=10000, batch_size=64, save_interval=1000):
    (X_train, _), (_, _) = dataset
    X_train = (X_train.astype(np.float32) - 127.5) / 127.5

    valid = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    for epoch in range(epochs):
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        imgs = X_train[idx]

        noise = np.random.normal(0, 1, (batch_size, 100))
        gen_imgs = generator.predict(noise)

        d_loss_real = discriminator.train_on_batch(imgs, valid)
        d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        g_loss = gan.train_on_batch(noise, valid)

        if epoch % save_interval == 0:
            print(f"{epoch} [D loss: {d_loss[0]} | D accuracy: {100*d_loss[1]}] [G loss: {g_loss}]")
            save_images(epoch, generator)

def save_images(epoch, generator, examples=10, dim=(1, 10), figsize=(10, 1)):
    noise = np.random.normal(0, 1, (examples, 100))
    generated_images = generator.predict(noise)
    generated_images = 0.5 * generated_images + 0.5

    plt.figure(figsize=figsize)
    for i in range(examples):
        plt.subplot(dim[0], dim[1], i+1)
        plt.imshow(generated_images[i])
        plt.axis('off')
    plt.tight_layout()
    plt.savefig(f"gan_generated_image_epoch_{epoch}.png")
    plt.close()

# 6. Transfer Learning with GANs on the CIFAR-10 Dataset
def transfer_learning_gan(dataset):
    generator = build_generator()
    discriminator = build_discriminator()
    gan = build_gan(generator, discriminator)
    train_gan(generator, discriminator, gan, dataset)

# Load CIFAR-10 dataset
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()

# Augment data for GAN training
X_train_augmented = augment_data(X_train)

# Build and train GAN
generator = build_generator()
discriminator = build_discriminator()
gan = build_gan(generator, discriminator)
train_gan(generator, discriminator, gan, (X_train_augmented, y_train))

# Transfer learning GAN
transfer_learning_gan((X_train_augmented, y_train))

This code covers:

Data augmentation for GAN training using random rotations, flipping, and cropping.
A simple discriminator model for classifying images as real or fake.
A generator model that uses transpose convolution to generate 32x32x3 images from random noise.
A Minimax loss function for GANs.
Combining the generator and discriminator into a GAN model and training it on the CIFAR-10 dataset.
Implementing transfer learning for GANs on the CIFAR-10 dataset.